In [3]:
!pip install jira
from jira import JIRA
import pandas as pd
from datetime import datetime, timedelta

# Configure your JIRA instance URL
JIRA_URL = 'https://ninjacart-tech.atlassian.net/'

# Define your JIRA username (email address) and API token
JIRA_USERNAME = 'rohtashrathore@ninjacart.com'
JIRA_API_TOKEN = 'ATATT3xFfGF0Ru3y1cM9ryaKNfwp5DR4ymYRvlqfeKrf_gFkhz_UFA4rZu40_o-xnOTzv-xWYoIk6924ZP1Cmahl3miDtPQTwxZAAvs6uiV8o5INVsbEj-4nkS-owk887fD56gy9BtnOa_H7ndJ7kj-ZQ5lSAyBNVdEjplrTrhFWaRTdw4a576Q=6C39855B'

# Connect to JIRA using API token authentication
jira = JIRA(JIRA_URL, basic_auth=(JIRA_USERNAME, JIRA_API_TOKEN))

# Function to parse disposition data
# Function to parse disposition data
def parse_disposition(data):
    if data:
        try:
            data_str = str(data)
            # Replace '->' with ' -> '
            data_str = data_str.replace("->", " -> ")
            # Replace '>' with ' -> '
            data_str = data_str.replace(">", " -> ")
            # Replace ' - ' with '->'
            data_str = data_str.replace(" - ", " -> ")
            return data_str.strip()
        except Exception:
            return str(data)
    else:
        return None

# Function to parse L3 Disposition data
def parse_l3_disposition(data):
    if data:
        try:
            data_str = str(data)
            # First, replace underscores with ' -> '
            data_str = data_str.replace("_*:*_", " -> ")
            # Then, replace '|' with '>'
            data_str = data_str.replace("|", ">")
            # Replace ' - ' with '->'
            data_str = data_str.replace(" - ", " -> ")
            return data_str
        except Exception:
            return str(data)
    else:
        return None


# Function to parse POD data
def parse_pod(data):
    if data:
        return data.replace("\n", " ")
    else:
        return None

# Function to fetch issues from a project and return as a DataFrame
def fetch_issues(project_key):
    # Calculate start and end dates for the last 45 days
    end_date = datetime.now()
    start_date = end_date - timedelta(days=60)

    # Format dates as strings in JIRA's preferred format (YYYY-MM-DD)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    # Construct JQL query with date range
    jql_query = f'project={project_key} AND created >= "{start_date_str}" AND created <= "{end_date_str}"'

    # Initialize list to store issues data
    data = []

    # Pagination variables
    start_at = 0
    max_results = 50

    # Parse custom field data
    def parse_custom_field(custom_field):
        if isinstance(custom_field, dict):
            return custom_field.get('value')
        elif isinstance(custom_field, list):
            return ', '.join([str(value.get('value')) for value in custom_field if value.get('value')])
        else:
            return custom_field

    # Fetch issues using pagination
    while True:
        issues = jira.search_issues(jql_query, startAt=start_at, maxResults=max_results)

        # If no more issues are returned, break the loop
        if not issues:
            break

        # Extract data from issues
        for issue in issues:
            issue_data = {
                'Issue key': issue.key,
                'Issue id': issue.id,
                'Summary': issue.fields.summary,
                'Assignee': getattr(issue.fields.assignee, 'displayName', None),
                'Assignee Id': getattr(issue.fields.assignee, 'key', None),
                'Status': issue.fields.status.name,
                'Resolution': getattr(issue.fields.resolution, 'name', None),
                'Created': issue.fields.created,
                'Updated': issue.fields.updated,
                'Creator': issue.fields.creator.displayName,
                'Creator Id': getattr(issue.fields.creator, 'key', None),
                'Custom field (POD)': issue.fields.customfield_10277,
                'Custom field (Call Type)': issue.fields.customfield_10283,
                'Custom field (Customer Number)': parse_custom_field(issue.fields.customfield_10273),
                'Custom field (Interaction Id )': parse_custom_field(issue.fields.customfield_10339),
                'Custom field (Query / Complaint)': parse_custom_field(issue.fields.customfield_10276),
                'Resolved': issue.fields.resolutiondate,
                'Custom field (Disposition)': parse_disposition(issue.fields.customfield_10278),
                'Custom field (User ID)': parse_custom_field(issue.fields.customfield_10300),
                'Custom field (Raised By)': parse_custom_field(issue.fields.customfield_10282),
                'Custom field (L3 Disposition)': parse_l3_disposition(issue.fields.customfield_10285),
                'Custom field (Mode Of Interaction )': getattr(issue.fields, 'customfield_10284', None),
                'Issue Type': issue.fields.issuetype.name,
                'Status Category': issue.fields.status.statusCategory.name
            }
            data.append(issue_data)


        # Increment start_at for the next pagination
        start_at += max_results

    # Create DataFrame from the collected data
    issues_df = pd.DataFrame(data)
    return issues_df

# Example usage
if __name__ == '__main__':
    project_key = 'TICKET'
    df4 = fetch_issues(project_key)
    df4.to_csv('output.csv', index=False)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.5 MB/s eta 0:00:00


In [5]:
import gspread
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import re
import io
import requests

# URL of the CSV data
url = "https://analytics-new-k8s.ninjacart.in/api/queries/20963/results.csv?api_key=91fEv7432lVyD8dm3jGPsUpyf3jra4UlBGaNG7Dt"

# Fetch the CSV data
response = requests.get(url)

# Check if request was successful
if response.status_code == 200:
    # Read the CSV data into a DataFrame
    df3 = pd.read_csv(io.StringIO(response.text))

#loading disposition file

# Open the Google Spreadsheet by its name (make sure you've shared it with the client email)
# gc = gspread.service_account(filename='D:\\json_file\\service_file.json')
from google.colab import drive
drive.mount('/content/drive')

# Use the file from Google Drive
import gspread

# Open the Google Spreadsheet by its name (make sure you've shared it with the client email)
gc = gspread.service_account(filename='/content/drive/My Drive/service_file.json')


# Open the Google Sheet
sh1 = gc.open('Dispositions_update')
worksheet1 = sh1.get_worksheet(0)

# Get the specific range of values (A1:H44)
data1 = worksheet1.get_values('A1:H44')

# Create a DataFrame with the retrieved data
dispo_master_df = pd.DataFrame(data1[1:], columns=data1[0])

sh2 = gc.open('Dispositions_update')
worksheet2 = sh2.get_worksheet(5)
data2 = worksheet2.get_all_values()
match_df = pd.DataFrame(data2[1:], columns=data2[0])


# Rename the columns
column_rename_map = {
    'L1 (High-Level Category)': 'L1',
    'L2 (Sub-Category)': 'L2',
    'L3 (Specific Issue)': 'L3'
}
dispo_master_df.rename(columns=column_rename_map, inplace=True)

# Create a copy of L2 as L3a
dispo_master_df['L3a'] = dispo_master_df['L2']

dispo_master_df['Key'] = dispo_master_df['POD'] + dispo_master_df['L1'] + dispo_master_df['L2'] + dispo_master_df['L3a'] + dispo_master_df['L3']
desired_order = ['Key','POD','L1', 'L2', 'L3', 'L3a','User','FTR/Non-FTR','Stream','TAT Breach/Tech Issue']
dispo_master_df = dispo_master_df[desired_order]
dispo_master_df.head()



df = df4.merge(df3, how='left', left_on='Custom field (User ID)', right_on='UserID')
df.head()

Mounted at /content/drive


,Issue key,Issue id,Summary,Assignee,Assignee Id,Status,Resolution,Created,Updated,Creator,...,KYC,Application,ChildLoan,ParentLoan,EDILoan,QR,BNPL,NTC,PG,Utilization %
0,TICKET-11717,143258,New NTC Onboarding || Page is loading for a lo...,Veeresh S Totar,None,PROD CLOSURE,Done,2024-06-17T18:58:41.337+0530,2024-06-18T10:27:33.572+0530,Alok Sulakhe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TICKET-11716,143218,blank chat,Priya D Bagalad,None,AGENT CLOSURE,Done,2024-06-17T17:13:40.066+0530,2024-06-17T17:15:03.895+0530,Kuriseti Sathwik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TICKET-11715,143210,Unable to upload CC BS ( M Lakshmi Kant),Dhanush K,None,PROD CLOSURE,Done,2024-06-17T16:57:40.346+0530,2024-06-17T17:22:27.954+0530,Priyanka Mehra,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,TICKET-11714,143196,Blank Call,Priya D Bagalad,None,AGENT CLOSURE,Done,2024-06-17T16:38:59.555+0530,2024-06-17T16:39:35.830+0530,Kuriseti Sathwik,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,TICKET-11713,143178,blank call,Priya D Bagalad,None,AGENT CLOSURE,Done,2024-06-17T16:21:41.183+0530,2024-06-17T18:40:24.326+0530,Kuriseti Sathwik,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
import numpy as np
from datetime import datetime
import re
# Parse date column after concatenating
df['Created'] = pd.to_datetime(df['Created'], dayfirst=True, format='%Y-%m-%dT%H:%M:%S.%f%z')
df['Resolved'] = pd.to_datetime(df['Resolved'], dayfirst=True, format='%Y-%m-%dT%H:%M:%S.%f%z')

df.drop_duplicates(subset=['Issue id'], keep='first', inplace=True)
filter = ~df['Summary'].str.contains(re.compile(r'test', re.IGNORECASE)).fillna(False)
df = df[filter]


# List of datetime columns
datetime_cols = ['Created', 'Updated', 'Resolved']

# Parse datetime columns
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col], dayfirst=True, format='%Y-%m-%dT%H:%M:%S.%f%z')


# Add date and week number columns
df['Created_Date'] = df['Created'].dt.date
df['Resolved_date'] = df['Resolved'].dt.date
df['Resolved_Week_'] = df['Resolved'].dt.isocalendar().week
df['Created_Week_'] = df['Created'].dt.isocalendar().week


df['Resolved_Week'] = df['Resolved'].dt.to_period('W').dt.to_timestamp()
df['Created_Week'] = df['Created'].dt.to_period('W').dt.to_timestamp()

split_cols = {
    'Custom field (Disposition)': ['L1', 'L2'],
    'Custom field (L3 Disposition)': ['L3a', 'L3'],
    'Custom field (Query / Complaint)': ['QRC', 'FCR/non-FCR'],
    'Custom field (Mode Of Interaction )': ['Source', 'Whatsapp_group_name'],
    'Custom field (Raised By)': ['Raised_by_L1', 'Raised_by_L2']
}

for col, new_cols in split_cols.items():
    # Convert column to string type
    df[col] = df[col].astype(str)
    # Replace second '->' with ' - ' if present
    df[col] = df[col].str.replace(' -> ', ' - ', n=1)
    # Split the modified column
    split_df = df[col].str.split(' - ', n=1, expand=True)
    # Check if the number of generated columns matches the number of specified new columns
    if len(new_cols) == 2:
        if len(split_df.columns) == 2:  # If splitting generates exactly two parts
            df[new_cols[0]] = split_df[0].str.strip()
            df[new_cols[1]] = split_df[1].str.strip()
        elif len(split_df.columns) == 1:  # If splitting generates only one part
            df[new_cols[0]] = split_df[0].str.strip()
            df[new_cols[1]] = None  # Assign None or NaN to the second new column
        else:
            raise ValueError("Unexpected error occurred while splitting the column.")
    else:
        raise ValueError("The number of new columns specified must be 2.")





# Fill NA
df = df.fillna({'FCR/non-FCR': 'FTR'})

#CORRECTIONS
df['L1'] = df['L1'].replace('KYC', 'KYC-related issues')
df['L1'] = df['L1'].replace('Loans', 'Loan-related issues')
df['L1'] = df['L1'].replace('QR Related', 'QR-related issues')
df['L1'] = df['L1'].replace('VAN /Payments related', 'VAN/Payment-related issues')
df['L2'] = df['L2'].replace('Enach Esign', 'E-sign/E-Nach')
df['L3'] = df['L3'].replace('Payment done -> limit not released', 'Payment done - limit not released')
df['L3'] = df['L3'].replace('Seller not onboarded/migrated -> visible on App', 'Seller not onboarded/migrated - visible on App')
df['L3'] = df['L3'].replace('Buyer not onboarded/migrated -> visible on App', 'Buyer not onboarded/migrated - visible on App')
df['L3'] = df['L3'].replace('EDI -> Insufficient Balance/Amount not getting deducted', 'EDI - Insufficient Balance/Amount not getting deducted')
df['L3'] = df['L3'].replace('Unable to Repay -> Repayment option not active', 'Unable to Repay - Repayment option not active')
df['L3'] = df['L3'].replace('Unable to create TF/BNPL loan -> KYC Pending', 'Unable to create TF/BNPL loan - KYC Pending')


# Replace NaN values with empty strings
import pandas as pd

def trim_strings(x):
    if pd.isna(x):
        return x
    else:
        return str(x).strip()

df.rename(columns={'Custom field (POD)': 'Pod'}, inplace=True)

# List of columns to trim
string_cols = ['L1', 'L2', 'L3a', 'L3', 'Pod']

# Convert columns to strings
for col in string_cols:
    df[col] = df[col].astype(str)

# Trim whitespace
for col in string_cols:
    df[col] = df[col].apply(trim_strings)
# Add flag columns
df['Resolution_flag'] = np.where(df['Status Category'].isin(['Done']), '1', '0')
df['Task_status'] = np.where(df['Status Category'].isin(['Done']), 'Closed', 'Open')
df['Task/subtask'] = np.where(df['Issue Type']=='Task', 1, 0)
df['FCR'] = np.where(df['QRC']=='Complaint', 0, 1)
df['Prod/Credit_Ops_Closure'] = np.where(df['Status'].isin(['Credit/Fin Ops','PROD CLOSURE']), 1, 0)
df['CMT/Agent_Closure'] = np.where(df['Status'].isin(['CMT CLOSURE','AGENT CLOSURE']), 1, 0)
#correct_dispositions
df['concatenated'] = df['Pod'] + df['L1'] + df['L2'] + df['L3a'] + df['L3']

df['correct_dispositions_check'] = df['concatenated'].apply(lambda x: 1 if x in dispo_master_df['Key'].values else 0)


# SLA buckets
df['SLA'] = (df['Resolved'] - df['Created']) / np.timedelta64(1, 'h')
df['SLA_Flag'] = pd.cut(df['SLA'], bins=[-1, 4, 8, 12, 24, 48, np.inf], labels=['0-4h', '4-8h', '8-12h', '12-24h', '24-48h', '48h+'])

# Calculate open SLA
# Convert 'current_time' to a timezone naive datetime

from datetime import datetime, timezone

# Calculate open SLA
# Convert 'current_time' to a timezone aware datetime in UTC
current_time = datetime.now(timezone.utc)
# Convert 'Created' column to a timezone naive datetime
df['Created'] = df['Created'].dt.tz_localize(None)
# Convert 'current_time' back to a timezone naive datetime
current_time = datetime.now().replace(tzinfo=None)

current_time = datetime.now().replace(tzinfo=None)
current_time = datetime.now(timezone.utc)
df['Created'] = df['Created'].dt.tz_localize(None)
current_time = datetime.now().replace(tzinfo=None)

df['open_sla'] = np.where(df['Task_status'] == 'Open', (current_time - df['Created']) / np.timedelta64(1, 'h'), np.nan)

# Combine SLA for open and closed tasks
df['total_sla'] = np.where(df['Task_status'] == 'Open', df['open_sla'], df['SLA'])


#-----------------------------------------------------------------------------------------------------------
#user_id_check
def check_correct_user(row, user_ids):
    pod = row['Pod']
    L3 = row['L3']
    user_id = row['UserID']

    if user_id in (-1,-13,0) or pod in ['CX'] :
        return 1

#     if user_id not in user_table.index:
#         return 0
    # Check if the UserID is NaN
    if pd.isna(user_id):
        return 0

    # Retrieve the user data for the current row
    user_row = user_ids.loc[user_id]

    if pod.strip() in ['Neobank', 'Platform']:
        return 1
    elif pod == 'NTC' and user_row['NTC'] == 1 and L3 in ['Approved but not disbursed', 'Something went wrong/Technical Issues', 'Unable to accept contract - DPD/Limit Issue', 'Trade Loan issues', 'Unable to create order', 'Seller not onboarded/migrated - visible on App', 'Buyer not onboarded/migrated - visible on App', 'NTC icon not visible', 'Payment done - limit not released', 'Transaction pending/failure', 'Unable to Repay - Repayment option not active', 'Unable to Repay - UPI/NEFT not available', 'Miscellaneous issues',
                                                          'Unable to accept contract', 'Unable to create/close contract', 'Unable to Repay','Unable to create TF/BNPL loan - KYC Pending']:
        return 1
    elif pod == 'Commerce' and (user_row['BNPL'] == 1 or user_row['PG'] == 1 or user_row['NTC']):
        return 1
    elif pod == 'Credit/Ops' and user_row['Application'] == 1 and L3 in ['E-sign not triggered/link expired', 'E-sign done, not reflecting', 'E-Nach related issues', 'E-sign not triggered/link expired/reflecting']:
        return 1
    elif pod == 'Lending' and user_row['Application'] == 1 and L3 in ['SOA/NOC Required','Applied but docs not collected','Approved but not disbursed','Docs collected but not approved','Rejected but no clarity on reason','Miscellaneous issues']:
        return 1
    elif pod == 'Lending' and user_row['ChildLoan'] == 1 and L3 in ['Payment done - limit not released','Transaction pending/failure','Unable to Repay - Repayment option not active','Unable to Repay - UPI/NEFT not available','EDI - Insufficient Balance/Amount not getting deducted','Loan paid but shown Active/Overdue','Unable to Repay']:
        return 1
    elif L3 in ['Withdrawal/Deposit for repayment','Main/VAN Account Addition','Seller not onboarded/migrated - visible on App','Buyer not onboarded/migrated - visible on App','Document deletion','Error while uploading','Name/Contact number change','How to complete KYC','Approval delay','KYC Pending/Rejected/Verification failed','Rejection reason','Main bank account addition']:
        return 1
    else:
        return 0
#--------------------------------------------------------------------
import pandas as pd
import io
import requests

# URL of the CSV data
url = "https://analytics-new-k8s.ninjacart.in/api/queries/20963/results.csv?api_key=91fEv7432lVyD8dm3jGPsUpyf3jra4UlBGaNG7Dt"

# Fetch the CSV data
response = requests.get(url)

# Check if request was successful
if response.status_code == 200:
    # Read the CSV data into a DataFrame
    user_ids = pd.read_csv(io.StringIO(response.text))

    # Load your user data from a CSV file
user_ids = user_ids.set_index('UserID')  # Make sure UserID is the index
df['correct_user'] = df.apply(lambda row: check_correct_user(row, user_ids), axis=1)

#---------------------------------------------------------------------------------------------------------
#Ftr_Check
def check_FTR(row, dispo_master_df):
    key_value = row['concatenated']  # Assuming "Key" is named as "concatenated" in jira_data
    FCR = row['FCR']

    if FCR == 'Follow-up':
        return 1

    if key_value in dispo_master_df['Key'].values:
        dispo_row = dispo_master_df[dispo_master_df['Key'] == key_value].iloc[0]

        if dispo_row['FTR/Non-FTR'] == 1 and FCR == 1:
            return 1
        elif dispo_row['FTR/Non-FTR'] == 1 and FCR == 0:
            return 0
        elif dispo_row['FTR/Non-FTR'] == 0 and FCR == 1:
            return 0
        elif dispo_row['FTR/Non-FTR'] == 0 and FCR == 0:
            return 1
        elif dispo_row['FTR/Non-FTR'] == 2 and FCR in [0, 1]:
            return 1
        else:
            return 1
    else:
        return 0

 # Return 0 if there is no match

# Assuming dispo_master is your disposition master DataFrame
df['FTR_check'] = df.apply(lambda row: check_FTR(row, dispo_master_df), axis=1)

df['L1_L2'] = df['L1'].astype(str) + '_' + df['L2'].astype(str)
df['L2_L3'] = df['L2'].astype(str) + '_' + df['L3'].astype(str)
df['L1_L2_L3'] = df['L1'].astype(str) + '_' + df['L2'].astype(str) + '_' + df['L3'].astype(str)
df['Pod_L1'] = df['Pod'].astype(str) + '_' + df['L1'].astype(str)
df['resolved_same_week'] = (df['Resolved_Week'] == df['Created_Week']).astype(int)



df['SLA_HH_MM'] = df['total_sla'].apply(lambda x: str(timedelta(hours=x)).split('.')[0])

# Select relevant columns
columns = [
    'Issue key',
    'Issue id',
    'Custom field (User ID)',
    'Creator',
    'Assignee',
    'Status',
    'Status Category',
    'Issue Type',
    'Created_Date',
    'Created_Week_',
    'Pod',
    'L1',
    'L2',
    'L3a',
    'L3',
    'QRC',
    'FCR/non-FCR',
    'Source',
    'Raised_by_L1',
    'Raised_by_L2',
    'Task_status',
    'Resolution_flag',
    'FCR',
    'Prod/Credit_Ops_Closure',
    'CMT/Agent_Closure',
    'SLA_Flag',
    'total_sla',
    'Name','Market','BU','CH','ZH','RH',
    'Resolved_date',
    'correct_user',
    'FTR_check',
    'correct_dispositions_check',
    'L1_L2',
    'L1_L2_L3',
    'Resolved_Week_',
    'Pod_L1',
    'resolved_same_week',
    'SLA_HH_MM'
    ]
df = df[columns]
df.head()

<ipython-input-6-853e64683001>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col], dayfirst=True, format='%Y-%m-%dT%H:%M:%S.%f%z')
<ipython-input-6-853e64683001>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col], dayfirst=True, format='%Y-%m-%dT%H:%M:%S.%f%z')
<ipython-input-6-853e64683001>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,Issue key,Issue id,Custom field (User ID),Creator,Assignee,Status,Status Category,Issue Type,Created_Date,Created_Week_,...,Resolved_date,correct_user,FTR_check,correct_dispositions_check,L1_L2,L1_L2_L3,Resolved_Week_,Pod_L1,resolved_same_week,SLA_HH_MM
0,TICKET-11717,143258,-12518891.0,Alok Sulakhe,Veeresh S Totar,PROD CLOSURE,Done,Task,2024-06-17,25,...,2024-06-18,0,0,0,NTC_Onboarding Issues,NTC_Onboarding Issues_App loading issue,25,NTC_NTC,1,15:28:52
1,TICKET-11716,143218,2051690.0,Kuriseti Sathwik,Priya D Bagalad,AGENT CLOSURE,Done,Task,2024-06-17,25,...,2024-06-17,1,1,1,Non-Value Added Calls_Blank call/Test call/ADC,Non-Value Added Calls_Blank call/Test call/ADC...,25,CX_Non-Value Added Calls,1,0:01:23
2,TICKET-11715,143210,2528147.0,Priyanka Mehra,Dhanush K,PROD CLOSURE,Done,Task,2024-06-17,25,...,2024-06-17,0,0,0,NTC_Onboarding Issues,NTC_Onboarding Issues_Miscellaneous issues,25,NTC_NTC,1,0:24:47
3,TICKET-11714,143196,1.0,Kuriseti Sathwik,Priya D Bagalad,AGENT CLOSURE,Done,Task,2024-06-17,25,...,2024-06-17,1,1,1,Non-Value Added Calls_Blank call/Test call/ADC,Non-Value Added Calls_Blank call/Test call/ADC...,25,CX_Non-Value Added Calls,1,0:00:36
4,TICKET-11713,143178,1.0,Kuriseti Sathwik,Priya D Bagalad,AGENT CLOSURE,Done,Task,2024-06-17,25,...,2024-06-17,1,1,1,Non-Value Added Calls_Blank call/Test call/ADC,Non-Value Added Calls_Blank call/Test call/ADC...,25,CX_Non-Value Added Calls,1,2:18:43


In [7]:
df.rename(columns = {'Custom field (User ID)':'user_id'}, inplace = True)



# Convert 'Created_Date' to datetime format
import pandas as pd
from datetime import datetime, timedelta

# Convert 'Resolved_date' column to datetime
df['Created_Date'] = pd.to_datetime(df['Created_Date'], errors='coerce')
# Condition 1: Filter data for the last 'x' days
start_date = (datetime.now() - timedelta(days=60)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

df_last_14_days = df[(df['Created_Date'] >= start_date) & (df['Created_Date'] <= end_date)]



df_last_14_days.to_csv('deep_dive_raw.csv')



In [ ]:
# df.fillna('NA', inplace=True)
# Save cleaned data
import gspread
from gspread_dataframe import set_with_dataframe

# Use the JSON key file to authenticate and create a client
# gc = gspread.service_account(filename='D:\\json_file\\service_file.json')

# Open the Google Spreadsheet by its name
sh = gc.open('Dashboard <> CX <> Reseller')

# Select the first worksheet in the spreadsheet
worksheet = sh.get_worksheet(5)


worksheet.clear()

# Write the DataFrame to the worksheet
set_with_dataframe(worksheet, df_last_14_days)


In [9]:
import gspread
from gspread_dataframe import set_with_dataframe

# Use the JSON key file to authenticate and create a client
# gc = gspread.service_account(filename='D:\\json_file\\service_file.json')

# Open the Google Spreadsheet by its name (make sure you've shared it with the client email)
sh = gc.open('datastudio_raw_file')


# Select the first worksheet in the spreadsheet
worksheet = sh.get_worksheet(0)

worksheet.clear()

# Write the DataFrame to the worksheet
set_with_dataframe(worksheet, df_last_14_days)

In [ ]:
import gspread
from gspread_dataframe import set_with_dataframe

# Use the JSON key file to authenticate and create a client
# gc = gspread.service_account(filename='D:\\json_file\\service_file.json')

# Open the Google Spreadsheet by its name (make sure you've shared it with the client email)
sh = gc.open('Defect_Elimination_meeting_data')


# Select the first worksheet in the spreadsheet
worksheet = sh.get_worksheet(0)


# Write the DataFrame to the worksheet
set_with_dataframe(worksheet, df_last_14_days)